<a href="https://colab.research.google.com/github/Zane-Z/RSM8521_Final_Project/blob/main/RSM8521_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install transformers
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
     |████████████████████████████████| 77 kB 2.3 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 
     |████████████████████████████████| 596 kB 14.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import uniform, randint
import torch
from transformers import pipeline
import argparse
import os

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

## Data Scraping and Agregating

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
from itertools import zip_longest
import bs4 as bs
import pickle
import requests
import wrds

import yfinance as yf
import datetime
import io
import warnings
import random
import numpy as np

### Getting S&P tickers

In [ ]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker[:-1])
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers
TickerList = save_sp500_tickers()

### Getting stock prices

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning) 

start = datetime.datetime(2016,2,1)
end = datetime.datetime(2022,4,12)

stock_final = pd.DataFrame()
# iterate over each symbol
for i in TickerList:  
    
    # print the symbol which is being downloaded
    print( str(TickerList.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None

In [ ]:
#stock_final['Date'] = stock_final.index
stock_final = stock_final.reset_index(level=0)
stock_final = stock_final[['Date','Name','Open','Close']]
stock_final.rename(columns = {'Name':'Ticker'}, inplace = True)

In [ ]:
stock_final['1Week'] = stock_final['Date']
stock_final['1Month'] = stock_final['Date']
stock_final['1Quarter'] = stock_final['Date']
stock_final['1Day'] = stock_final['Date']

for i in range(1,31):
    name = 'Hist' + str(i)
    stock_final[name] = stock_final['Date']
stock_final

In [ ]:
# List of Tickers already completed
try:
    Done = pd.read_csv('Transcripts.csv')['Ticker'].unique()
except:
    Done = []


### Scraping Seeking Alpha

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning) 

Tickers = []
Transcripts = []
df = pd.DataFrame(columns = ['Ticker','Transcript','Date'])

#driver = webdriver.Chrome()

for Ticker in TickerList:
    if Ticker not in Done:

        print(Ticker)

        gotoPre2 = 'https://seekingalpha.com'
        gotoPre = 'https://seekingalpha.com/symbol/' + str(Ticker)
        goto = 'https://seekingalpha.com/symbol/' + str(Ticker) + '/earnings/transcripts'


        #time.sleep(5)
        #time.sleep(random.randint(1,5))

        option = webdriver.ChromeOptions()
        option.add_argument('--disable-blink-features=AutomationControlled')
        option.add_argument("window-size=720,400")
        option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
        driver = webdriver.Chrome(options = option)

        #time.sleep(1)

        driver.delete_all_cookies()
        driver.get(gotoPre2)
        time.sleep(1)
        driver.get(gotoPre)
        time.sleep(1)
        driver.get(goto)
        time.sleep(1)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        links = []

        for a in soup.find_all('a', string= re.compile('Call Transcript')):
            #print('Found the URL:', a['href'])
            links.append(a['href'])
        
    
        #time.sleep(2)

        for link in links:
            test = 'https://seekingalpha.com/' + link
            #print(link)
            print(len(Transcripts))
            driver.delete_all_cookies()
            driver.get(goto)
            #time.sleep(1)
            driver.get(test)
            driver.delete_all_cookies()
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            for item in soup.find_all("span", {"data-test-id":"post-date"}):
                Date = str(item.getText())

            for item in soup.find_all("div", {"data-test-id":"content-container"}):
                try:
                    if len(item.getText()) > 5000:

                        Text = str(item.getText())

                        Transcripts.append(Text)
                        Tickers.append(Ticker)

                        series = pd.Series([Ticker,Text,Date], index = df.columns)
                        df = df.append(series, ignore_index=True)

                except:
                    pass
        driver.close()

### Data Agregation

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

values = [0, 2, 1]
values2 = [0, 1, 2]

conditions = []
for i in range(0,91):
    conditions.append([
        ((df['Date'] + datetime.timedelta(days=i)).dt.day_of_week <= 4),
        ((df['Date'] + datetime.timedelta(days=i)).dt.day_of_week == 5),
        ((df['Date'] + datetime.timedelta(days=i)).dt.day_of_week == 6)
        ])

conditions2 = []
for k in range(0,31):
    conditions2.append([
        ((df['Date'] - datetime.timedelta(days=k)).dt.day_of_week <= 4),
        ((df['Date'] - datetime.timedelta(days=k)).dt.day_of_week == 5),
        ((df['Date'] - datetime.timedelta(days=k)).dt.day_of_week == 6)
        ])


for i in range(0,len(conditions)):
    name = 'Offset'+str(i)
    df[name] = np.select(conditions[i], values)

for i in range(0,len(conditions2)):
    name = 'Offset_Neg' + str(i)
    df[name] = np.select(conditions2[i], values2)

df['Date'] = pd.to_datetime(df['Date']).dt.date
df['1Day'] = df['Date'] + datetime.timedelta(days=1) +  df['Offset1'].map(datetime.timedelta)
df['1Week'] = df['Date'] + datetime.timedelta(days=7) + df['Offset7'].map(datetime.timedelta)
df['1Month'] = df['Date'] + datetime.timedelta(days=30) + df['Offset30'].map(datetime.timedelta)
df['1Quarter'] = df['Date'] + datetime.timedelta(days=90) + df['Offset90'].map(datetime.timedelta)

df[['Date','1Day', '1Week', '1Month', '1Quarter']] = df[['Date','1Day', '1Week', '1Month', '1Quarter']].apply(pd.to_datetime)

for i in range(1,31):
    name = 'Hist' + str(i)
    OffsetName = 'Offset_Neg' + str(i)
    df[name] = df['Date'] + datetime.timedelta(days = -i) - df[OffsetName].map(datetime.timedelta)
    df[name] = df[name].apply(pd.to_datetime)

In [ ]:
df2 = df.merge(stock_final, on = ['Ticker','Date'], how = 'left',suffixes=('', '_Current'))
df2 = df2.merge(stock_final, on = ['Ticker','1Day'], how = 'left',suffixes=('', '_1Day'))
df2 = df2.merge(stock_final, on = ['Ticker','1Week'], how = 'left',suffixes=('', '_1Week'))
df2 = df2.merge(stock_final, on = ['Ticker','1Month'], how = 'left',suffixes=('', '_1Month'))
df2 = df2.merge(stock_final, on = ['Ticker','1Quarter'], how = 'left',suffixes=('', '_1Quarter'))

for i in range(1,31):
    name = 'Hist' + str(i)
    df2 = df2.merge(stock_final, on = ['Ticker',name], how = 'left',suffixes=('', '_'+ name))


for column in df2.columns:
    if any(x in column for x in ['_1Week','_1Quarter', '_1Month', '_1Day']):
        if 'Close' not in column:
            df2.drop(column, axis=1, inplace=True)

for column in df2.columns:
    if any(x in column for x in ['_Current']):
        if 'Open' not in column:
            df2.drop(column, axis=1, inplace=True)


histList =[]
for i in range(0,31):
    histList.append('_Hist' + str(i))

for column in df2.columns:
    if any(x in column for x in histList):
        if 'Close' not in column:
            df2.drop(column, axis=1, inplace=True)

for column in df2.columns:
    if 'Offset' in column:
        df2.drop(column, axis=1, inplace=True)

In [ ]:

df2['1WeekChange'] = (df2['Close_1Week'] - df2['Open'])/ df2['Open']
df2['1MonthChange'] = (df2['Close_1Month'] - df2['Open'])/ df2['Open']
df2['1QuarterChange'] = (df2['Close_1Quarter'] - df2['Open'])/ df2['Open']
df2['1DayChange'] = (df2['Close_1Day'] - df2['Open'])/ df2['Open']

for k in range(1,31):
    name = 'Close_Hist' + str(k)
    df2[name] = df2[name]/df2['Open']

pd.set_option('display.max_columns', 100)
df2

In [ ]:
#df.to_csv('Transcripts.csv', index = False)

# if file does not exist write header 
if not os.path.isfile('Transcripts.csv'):
   df2.to_csv('Transcripts.csv', header=df2.columns, index = False)
else: # else it exists so append without writing the header
   df2.to_csv('Transcripts.csv', mode='a', header=False, index = False)

## Generating Sentiments

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Set seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
# This will ask you to give Colab permission to access your Google Drive 
# and enter an authorization code.
# See: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# MODIFY THIS LINE with the path to where you saved the datafiles on your Google drive

path = '/content/drive/MyDrive/DeepLearningProject/'

In [ ]:
## Read Transcripts

data_path = path + 'Transcripts.csv'
transcripts = pd.read_csv(data_path)

train, test = train_test_split(transcripts, test_size=0.2, random_state=RANDOM_SEED)
train, validation = train_test_split(train, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
def splitter(n, s):
    pieces = s.split()
    return (" ".join(pieces[i:i+n]) for i in range(0, len(pieces), n))

In [ ]:
def predict(text):

    neg = []
    neutral = []
    pos = []
    for piece in splitter(250, text):

        inputs = tokenizer(piece, return_tensors="pt")

        try:

            with torch.no_grad():
                logits = model(**inputs).logits

            predicted_class_id = logits.argmax().item()

            neg.append(logits.tolist()[0][0])
            neutral.append(logits.tolist()[0][1])
            pos.append(logits.tolist()[0][2])
        except:
            neg.append(0)
            neutral.append(0)
            pos.append(0)
    
    Sentiment = [np.average(neg), np.average(neutral), np.average(pos)]

    return Sentiment

In [ ]:
Samples = test[0:100]
sentiments = []

for Sample in Samples['Transcript']:
    prediction = predict(Sample)
    sentiments.append(prediction)

Samples['SentimentsPos'] = [i[0] for i in sentiments]
Samples['SentimentsNeutral'] = [i[1] for i in sentiments]
Samples['SentimentsNeg'] = [i[2] for i in sentiments]

Samples.to_csv('Out5.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

## Linear Regression

In [ ]:
data_path = path + 'Transcripts.csv'
transcripts = pd.read_csv(data_path)

data_path = path + 'Out2.csv'
Out = pd.read_csv(data_path)

data_path = path + 'Out3.csv'
Out2 = pd.read_csv(data_path)

data_path = path + 'Out4.csv'
Out3 = pd.read_csv(data_path)

In [ ]:
Out = Out[['Ticker','Date','SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg']]
Out2 = Out2[['Ticker','Date','SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg']]
Out3 = Out3[['Ticker','Date','SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg']]

In [ ]:
TranscriptsOut1  = pd.merge(transcripts, Out, on = ['Ticker','Date'])
TranscriptsOut2  = pd.merge(transcripts, Out2, on = ['Ticker','Date'])
TranscriptsOut3  = pd.merge(transcripts, Out3, on = ['Ticker','Date'])

TranscriptsOut = TranscriptsOut1.append(TranscriptsOut2)
TranscriptsOut = TranscriptsOut.append(TranscriptsOut3)

TranscriptsOut.to_csv('TranscriptsWithSentiment.csv')

In [ ]:
# Set seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
## Read Transcripts

#data_path = path + 'Transcripts.csv'
#transcripts = pd.read_csv(data_path)

train, test = train_test_split(TranscriptsOut, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
import statsmodels.api as sm
from scipy import stats

testSpan = '1QuarterChange'

Modeling = TranscriptsOut[TranscriptsOut[testSpan].isnull() == False]

X = Modeling[['SentimentsPos', 'SentimentsNeg', 'SentimentsNeutral']]
y = Modeling[testSpan]

X['MaxCol'] = X.max(axis=1)
X['NegDummy'] =  np.where(X['SentimentsNeg']== X['MaxCol'], 1, 0)
X['NeutralDummy']  = np.where(X['SentimentsNeutral']== X['MaxCol'], 1, 0)
X['PosDummy'] =  np.where(X['SentimentsPos']== X['MaxCol'], 1, 0)

X = X[['NegDummy', 'PosDummy']]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         1QuarterChange   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9148
Date:                Sun, 17 Apr 2022   Prob (F-statistic):              0.402
Time:                        16:20:36   Log-Likelihood:                 137.12
No. Observations:                 230   AIC:                            -268.2
Df Residuals:                     227   BIC:                            -257.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0320      0.007      4.622      0.0

In [ ]:
Modeling2 = test[test[testSpan].isnull() == False]

X2 = Modeling2[['SentimentsPos', 'SentimentsNeg', 'SentimentsNeutral']]
y2 = Modeling2[testSpan]

X2['MaxCol'] = X2.max(axis=1)
X2['NegDummy'] =  np.where(X2['SentimentsNeg']== X2['MaxCol'], 1, 0)
X2['NeutralDummy']  = np.where(X2['SentimentsNeutral']== X2['MaxCol'], 1, 0)
X2['PosDummy'] =  np.where(X2['SentimentsPos']== X2['MaxCol'], 1, 0)

X2 = X2[['NegDummy', 'PosDummy']]
X3 = sm.add_constant(X2)

Ypred = list(est2.predict(X3))

sklearn.metrics.mean_absolute_error(y2,Ypred)

0.02720490279139881

## ANN

In [ ]:
# Set seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
# This will ask you to give Colab permission to access your Google Drive 
# and enter an authorization code.
# See: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# MODIFY THIS LINE with the path to where you saved the datafiles on your Google drive

path = '/content/drive/MyDrive/RSM 8521 Deep Learning/Final Project/'

In [ ]:
# Reading data
# python scripts/datasets.py --data_path <path to Sentences_50Agree.txt>


data_path = path + '/FinancialPhraseBank-v1.0/Sentences_50Agree.txt'

# Train/Validation Split
# parser = argparse.ArgumentParser(description='Sentiment analyzer')
# parser.add_argument('train_path', type=str, help='Path to the text file.')
# args = parser.parse_args()

data = pd.read_csv(data_path, encoding = "ISO-8859-1", sep='.@', names=['text','label'])

train, test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)
train, validation = train_test_split(train, test_size=0.2, random_state=RANDOM_SEED)

# train.to_csv('data/sentiment_data/train.csv',sep='\t')
# test.to_csv('data/sentiment_data/test.csv',sep='\t')
# valid.to_csv('data/sentiment_data/validation.csv',sep='\t')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
train.head()

,text,label
3618,"In 2008 , AVC Systemhaus had net sales of EUR ...",neutral
1630,The device offers multimedia performance along...,neutral
4303,After Chuck Smith was laid off on May 30 from ...,negative
2575,Suomen Paikallissanomat Oy is part of Alma Med...,neutral
3914,The through service of FESCO is a telling exam...,positive


In [ ]:
train.shape

(3100, 2)

In [ ]:
train.loc[0, 'text']

'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing '

In [ ]:
train.loc[0, 'label']

'neutral'

In [ ]:
# Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
def predict(text):
  inputs = tokenizer(text, return_tensors="pt")

  with torch.no_grad():
      logits = model(**inputs).logits

  predicted_class_id = logits.argmax().item()
  return logits, model.config.id2label[predicted_class_id]

In [ ]:
sample_txt = 'Guest spending at our domestic parks has been exceptionally strong, \
              with average spending up significantly versus fiscal 2019.'

In [ ]:
logits, outcome = predict(sample_txt)

In [ ]:
outcome

'positive'

In [ ]:
logits

tensor([[ 2.0998, -1.7574, -1.6564]])

In [ ]:
model.config.id2label[1]

'negative'

In [ ]:
# Test on the test dataset
wrong = 0
for i in range(train.shape[0]):
  logits, prediction = predict(train.iloc[i]['text'])
  if prediction != train.iloc[i]['label']:
    wrong += 1
print(wrong/train.shape[0])

0.10935483870967742


=========================== Linear Regression ===========================

In [ ]:
# model_lg = LinearRegression()
# reg = model_lg.fit(X, y)

# reg.score(X, y)

# reg.coef_

# reg.intercept_

# reg.predict(np.array([[3, 5]]))

NameError: ignored

=========================== ANN with Sentiment ===========================

In [ ]:
# Web scapped data
data_sentiment = pd.read_csv(path + 'TranscriptsWithSentiment.csv')
data_sentiment.head()

,Unnamed: 0,Ticker,Transcript,Date,1Week,1Month,1Quarter,Hist1,Hist2,Hist3,...,Close_Hist28,Close_Hist29,Close_Hist30,1WeekChange,1MonthChange,1QuarterChange,1DayChange,SentimentsPos,SentimentsNeutral,SentimentsNeg
0,0,MMM,3M Company (NYSE:MMM) Q3 2021 Earnings Confere...,2021-10-26,2021-11-02,2021-11-25,2022-01-24,2021-10-25,2021-10-22,2021-10-22,...,0.005492,0.005545,0.005522,0.004970,NaN,-0.045673,-0.015629,0.192924,-0.862446,0.445982
1,1,MMM,3M Company (NYSE:MMM) Q2 2021 Results Conferen...,2021-07-27,2021-08-03,2021-08-26,2021-10-25,2021-07-26,2021-07-23,2021-07-23,...,0.005040,0.005052,0.005007,0.019370,-0.009584,-0.075098,0.005426,0.303203,-0.804976,0.151326
2,2,AOS,A. O. Smith Corporation (NYSE:AOS) Q4 2020 Ear...,2021-01-28,2021-02-04,2021-03-01,2021-04-28,2021-01-27,2021-01-26,2021-01-25,...,0.017810,0.017774,0.017774,0.056957,0.093547,0.199351,-0.021269,0.177426,-1.017115,0.614343
3,3,AOS,A. O. Smith Corporation (NYSE:AOS) Q3 2020 Ear...,2020-10-29,2020-11-05,2020-11-30,2021-01-27,2020-10-28,2020-10-27,2020-10-26,...,0.019071,0.018790,0.019121,0.015280,0.062252,0.003396,-0.024901,0.102874,-0.971596,0.558406
4,4,AOS,A. O. Smith Corporation (NYSE:AOS) Q2 2020 Ear...,2020-07-30,2020-08-06,2020-08-31,2020-10-28,2020-07-29,2020-07-28,2020-07-27,...,0.017313,0.016940,0.017240,-0.070390,-0.063313,-0.015685,-0.079189,-0.250090,-0.432363,0.573945


In [ ]:
data_sentiment.shape

(204, 81)

In [ ]:
data_sentiment.columns.get_loc('Close_Hist1')

44

In [ ]:
# Preprocessing
# Extracting useful columns
data_sentiment_features = data_sentiment.iloc[:, 44:]
data_sentiment_features = data_sentiment_features.drop(columns=['1WeekChange', '1MonthChange', '1QuarterChange'])

# Replace NA with 0
data_sentiment_features = data_sentiment_features.replace(np.nan,0)

# Standardizing
# scaler = StandardScaler()
# data_sentiment_features_scaled = pd.DataFrame(scaler.fit_transform(data_sentiment_features), columns = data_sentiment_features.columns)
data_sentiment_features_scaled = data_sentiment_features

# Dummifying Sentiments
# data_sentiment_features_scaled['Positive'] = 0
# data_sentiment_features_scaled['Negative'] = 0
# for i in range(data_sentiment_features_scaled.shape[0]):
#   if data_sentiment_features_scaled.loc[i, 'SentimentsPos'] == max(data_sentiment_features_scaled.loc[i, 'SentimentsPos'], data_sentiment_features_scaled.loc[i, 'SentimentsNeutral'], data_sentiment_features_scaled.loc[i, 'SentimentsNeg']):
#     data_sentiment_features_scaled.loc[i, 'Positive'] = 1
#   elif data_sentiment_features_scaled.loc[i, 'SentimentsNeg'] == max(data_sentiment_features_scaled.loc[i, 'SentimentsPos'], data_sentiment_features_scaled.loc[i, 'SentimentsNeutral'], data_sentiment_features_scaled.loc[i, 'SentimentsNeg']):
#     data_sentiment_features_scaled.loc[i, 'Negative'] = 1
#   else:
#     pass

data_sentiment_features_scaled.head()

,Close_Hist1,Close_Hist2,Close_Hist3,Close_Hist4,Close_Hist5,Close_Hist6,Close_Hist7,Close_Hist8,Close_Hist9,Close_Hist10,...,Close_Hist25,Close_Hist26,Close_Hist27,Close_Hist28,Close_Hist29,Close_Hist30,1DayChange,SentimentsPos,SentimentsNeutral,SentimentsNeg
0,0.005563,0.005514,0.005514,0.005514,0.005555,0.005564,0.005561,0.005543,0.005549,0.005549,...,0.005389,0.005350,0.005517,0.005492,0.005545,0.005522,-0.015629,0.192924,-0.862446,0.445982
1,0.005185,0.005155,0.005155,0.005155,0.005119,0.005162,0.005164,0.005080,0.005126,0.005126,...,0.005140,0.005119,0.005107,0.005040,0.005052,0.005007,0.005426,0.303203,-0.804976,0.151326
2,0.017281,0.017703,0.018492,0.018765,0.018765,0.018765,0.018856,0.018791,0.018583,0.000000,...,0.000000,0.000000,0.000000,0.017810,0.017774,0.017774,-0.021269,0.177426,-1.017115,0.614343
3,0.018313,0.018871,0.019192,0.019548,0.019548,0.019548,0.019541,0.019505,0.019619,0.019558,...,0.019049,0.019049,0.019049,0.019071,0.018790,0.019121,-0.024901,0.102874,-0.971596,0.558406
4,0.019501,0.019113,0.019333,0.018773,0.018773,0.018773,0.018733,0.018667,0.018681,0.018477,...,0.000000,0.000000,0.000000,0.017313,0.016940,0.017240,-0.079189,-0.250090,-0.432363,0.573945


In [ ]:
data_sentiment_features_scaled.shape

(204, 34)

In [ ]:
y = data_sentiment_features_scaled['1WeekChange']
X = data_sentiment_features_scaled.drop(columns=['1DayChange']) # , 'SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

KeyError: ignored

In [ ]:
X_train.head()

In [ ]:
X_train.shape

(163, 33)

In [ ]:
max(y) - min(y)

0.3070226603707882

In [ ]:
# Combination of l2 and l1_l2
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(33,)))
                # , kernel_regularizer=regularizers.l2(1e-3), bias_regularizer=regularizers.l1_l2(l1 = 1e-3, l2 = 1e-3)))

model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))

model.add(Dense(1,))
model.compile(loss='mae', optimizer=Adam(learning_rate=0.001))

model.fit(X_train, y_train, epochs=200, verbose=1)

Epoch 1/200
6/6 [==============================] - 1s 5ms/step - loss: 0.3069
Epoch 2/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1897
Epoch 3/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1531
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1268
Epoch 5/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1216
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 0.1260
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 0.1018
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0929
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0950
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0869
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0837
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0759
Epoch 13/200
6/6 [==============================] - 0s 3ms/st

In [ ]:
mae = model.evaluate(X_train, y_train)

6/6 [==============================] - 0s 3ms/step - loss: 0.0300


In [ ]:
mae = model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.0333


=========================== ANN without Sentiment ===========================

In [ ]:
X_1 = data_sentiment_features_scaled.drop(columns=['1DayChange', 'SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg']) # , 'SentimentsPos', 'SentimentsNeutral', 'SentimentsNeg', 'Positive', 'Negative'
X_1_train, X_1_test = train_test_split(X_1, test_size=0.20)

In [ ]:
X_1_train.shape

(163, 30)

In [ ]:
# Combination of l2 and l1_l2
model1 = Sequential()
model1.add(Dense(32, activation='relu', input_shape=(30,)))
                # , kernel_regularizer=regularizers.l2(1e-3), bias_regularizer=regularizers.l1_l2(l1 = 1e-3, l2 = 1e-3)))

model1.add(Dropout(0.5))

model1.add(Dense(32, activation='relu'))

model1.add(Dropout(0.5))

model1.add(Dense(16, activation='relu'))

model1.add(Dense(1,))
model1.compile(loss='mae', optimizer=Adam(learning_rate=0.001))

model1.fit(X_1_train, y_train, epochs=200, verbose=1)

Epoch 1/200
6/6 [==============================] - 1s 3ms/step - loss: 0.0395
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0340
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0330
Epoch 4/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0312
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0319
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0320
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0311
Epoch 8/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0304
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0314
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0306
Epoch 11/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0312
Epoch 12/200
6/6 [==============================] - 0s 4ms/step - loss: 0.0301
Epoch 13/200
6/6 [==============================] - 0s 3ms/st

In [ ]:
mae = model1.evaluate(X_1_train, y_train)

6/6 [==============================] - 0s 2ms/step - loss: 0.0299


In [ ]:
mae = model1.evaluate(X_1_test, y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.0336


=========================== Compared to the mean error ===========================

In [ ]:
mean = np.mean(y_test)
mean

0.004774766367698131

In [ ]:
y_mean_df = pd.DataFrame(index=range(y_test.shape[0]),columns=range(1))
y_mean_df[0] = mean

In [ ]:
y_mean_df.shape

(41, 1)

In [ ]:
y_test.shape

(41,)

In [ ]:
mean_absolute_error(y_test, y_mean_df)

0.03369115859464349

=========================== Tokenizer in details and showcase ===========================

In [ ]:
# Showcase of the tokenizer
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: Guest spending at our domestic parks has been exceptionally strong,               with average spending up significantly versus fiscal 2019.
   Tokens: ['guest', 'spending', 'at', 'our', 'domestic', 'parks', 'has', 'been', 'exceptionally', 'strong', ',', 'with', 'average', 'spending', 'up', 'significantly', 'versus', 'fiscal', '2019', '.']
Token IDs: [4113, 5938, 2012, 2256, 4968, 6328, 2038, 2042, 17077, 2844, 1010, 2007, 2779, 5938, 2039, 6022, 6431, 10807, 10476, 1012]


**[SEP] - marker for ending of a sentence \
[CLS] - add this token to the start of each sentence, so BERT knows we’re doing classification \
[PAD] - special token for padding \
[UNK] - Everything else can be encoded using the [UNK] (unknown) token**

In [ ]:
# What the tokenizer does and the encoding
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Input IDs of encoding
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

32


tensor([  101,  4113,  5938,  2012,  2256,  4968,  6328,  2038,  2042, 17077,
         2844,  1010,  2007,  2779,  5938,  2039,  6022,  6431, 10807, 10476,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])

In [ ]:
# Attention Masks
print(len(encoding['attention_mask'][0]))
encoding['attention_mask']

32


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
# Convert IDs back to the original Tokens
# tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

=========================== Code Blocks that may be useful ===========================

In [ ]:
# token_lens = []
# for txt in df.content:
#   tokens = tokenizer.encode(txt, max_length=512)
#   token_lens.append(len(tokens))

In [ ]:
# classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
# classifier("I love you!")

In [ ]:
# num_labels = len(model.config.id2label)
# model = BertForSequenceClassification.from_pretrained(
#     "textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels, problem_type="multi_label_classification"
# )

# labels = torch.nn.functional.one_hot(torch.tensor([predicted_class_id]), num_classes=num_labels).to(
#     torch.float
# )
# loss = model(**inputs, labels=labels).loss
# loss.backward()